In [5]:
import pickle
import pandas as pd
import numpy as np
import pymorphy2
from multiprocessing import Pool
import re
from nltk.corpus import stopwords
from string import punctuation
from collections import Counter
from tqdm import tqdm_notebook   

import torch
import torch.nn as nn
import torch.nn.functional as F

%run ./func_vk.ipynb

In [6]:
seq_size=32
batch_size=16
embedding_size=64
lstm_size=64
gradients_norm=5
initial_words=['Есть']
predict_top_k=5
checkpoint_path='checkpoint'

In [7]:
with open('vk_df_habensk', 'rb') as f:
    df_post = pickle.load(f)

df_post

,text,id,date,user_id_comment,user_id_like,like_comment_id
0,"🎁Совсем скоро начнется предновогодняя суета, в...",14332,1604649611,"[527903014, 217810541, 530003372, 530003372, 5...","[587274, 60976715, 52111849, 223941449, 121039...",NaN
1,"#ЭТОНЕЛЕЧИТСЯ, #РАКЛЕЧИТСЯ — это наша всеросси...",14326,1604219234,"[534246999, 534246999, 466416995, 530003372]","[12208371, 98814822, 167109807, 46445691, 1372...",NaN
2,Прямо сейчас мы начинаем наш открытый диалог н...,14297,1604048939,"[217810541, 20520693, 530003372]","[167109807, 379409021, 6730100, 246658570, 700...",[311433276]
3,"Уже завтра, 30 октября, в рамках Международной...",14295,1603982535,[],"[167109807, 59369793, 43370298, 6450586, 97951...",NaN
4,"Осведомлен - значит вооружен! Да-да, именно эт...",14287,1603900038,[],"[167109807, 97951215, 46445691, 372858868, 323...",NaN
...,...,...,...,...,...,...
1585,"Ура, нам подарили мебель для оборудования 3 по...",8,1362135968,[],"[1215846, 80400611, 147631357, 201438128]",NaN
1586,Обращение Константина Хабенского:\n«Дорогие др...,4,1361290520,[3812093],"[2688481, 3121208, 3812093, 4251769, 5551925, ...",[116677333]
1587,"Друзья, история Насти Терлецкой – это тот случ...",3,1360814491,[],"[2688481, 5501348, 6030979, 116677333, 2096571...",NaN
1588,"Друзья, приветствуем вас на странице Благотвор...",2,1360595935,[146390022],[],NaN


In [8]:
df_post['like_comment_id'].fillna(value=0, inplace = True)
df_post['like_comment_id'] = df_post['like_comment_id'].apply(lambda x: fillna_with_arr(x))
df_post['like'] = (
    
    df_post['user_id_comment'].apply(lambda x: len(x)) 
    + df_post['user_id_like'].apply(lambda x: len(x)) 
    + df_post['like_comment_id'].apply(lambda x: len(x))
)

df_post['like_comment'] = df_post['user_id_comment'].apply(lambda x: len(x))
df_post['like_post'] = df_post['user_id_like'].apply(lambda x: len(x))
df_post['like_comment_comment'] = df_post['like_comment_id'].apply(lambda x: len(x))
clean_df(df_post)
df_post.head()

/home/alex/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


,text,id,date,user_id_comment,user_id_like,like_comment_id,like,like_comment,like_post,like_comment_comment,lemmas_text
0,"🎁Совсем скоро начнется предновогодняя суета, в...",14332,1604649611,"[527903014, 217810541, 530003372, 530003372, 5...","[587274, 60976715, 52111849, 223941449, 121039...",[],59,7,52,0,совсем скоро начаться предновогодний суета вре...
1,"#ЭТОНЕЛЕЧИТСЯ, #РАКЛЕЧИТСЯ — это наша всеросси...",14326,1604219234,"[534246999, 534246999, 466416995, 530003372]","[12208371, 98814822, 167109807, 46445691, 1372...",[],66,4,62,0,этонелечиться раклечиться это наш всероссийски...
2,Прямо сейчас мы начинаем наш открытый диалог н...,14297,1604048939,"[217810541, 20520693, 530003372]","[167109807, 379409021, 6730100, 246658570, 700...",[311433276],57,3,53,1,прямо начинать наш открытый диалог тема опухол...
3,"Уже завтра, 30 октября, в рамках Международной...",14295,1603982535,[],"[167109807, 59369793, 43370298, 6450586, 97951...",[],50,0,50,0,уже завтра октябрь рамка международный неделя ...
4,"Осведомлен - значит вооружен! Да-да, именно эт...",14287,1603900038,[],"[167109807, 97951215, 46445691, 372858868, 323...",[],70,0,70,0,осведомить значит вооружённый да именно этот ф...


Берем самые залайканные посты

In [40]:
# df_post[df_post.like > np.percentile(df_post['like'], 90)]

In [12]:
# df_post[df_post.like > np.percentile(df_post['like'], 90)]['text'].str.cat(sep = ' ')

In [23]:
# Сохраняем накачаное добро 
with open('vk_df_habensk', 'wb') as f:
    pickle.dump(df_post[df_post.like > np.percentile(df_post['like'], 90)]['text'].str.cat(sep = ' '), f)

In [3]:
with open('text_vk_legko_df', 'rb') as f:
    df = pickle.load(f)

In [14]:
# get_data_from_file('text_vk_legko_df', 16, 32)

In [15]:
int_to_vocab, vocab_to_int, n_vocab, in_text, out_text = get_data_from_file('text_vk_podari_df', 16, 32)

In [16]:
# int_to_vocab

In [17]:
# vocab_to_int

In [18]:
n_vocab

10352

In [19]:
# in_text

In [20]:
# out_text

In [21]:
# next(get_batches(in_text=in_text, out_text=out_text, batch_size=batch_size, seq_size=seq_size))

In [22]:
train_file = 'text_vk_legko_df'

In [23]:
class RNNModule(nn.Module):
    def __init__(self, n_vocab, seq_size, embedding_size, lstm_size):
        super(RNNModule, self).__init__()
        self.seq_size = seq_size
        self.lstm_size = lstm_size
        self.embedding = nn.Embedding(n_vocab, embedding_size)
        self.lstm = nn.LSTM(embedding_size,
                            lstm_size,
                            batch_first=True)
        self.dense = nn.Linear(lstm_size, n_vocab)
        
    def forward(self, x, prev_state):
        embed = self.embedding(x)
        output, state = self.lstm(embed, prev_state)
        logits = self.dense(output)

        return logits, state
    
    def zero_state(self, batch_size):
        return (torch.zeros(1, batch_size, self.lstm_size),
                torch.zeros(1, batch_size, self.lstm_size))
    
    

In [28]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
int_to_vocab, vocab_to_int, n_vocab, in_text, out_text = get_data_from_file(
    'text_vk_legko_df', batch_size, seq_size)

net = RNNModule(n_vocab, seq_size,
                embedding_size, lstm_size)
net = net.to(device)

criterion, optimizer = get_loss_and_train_op(net, 0.01)

iteration = 0

In [29]:
for e in range(50):
    batches = get_batches(in_text, out_text, batch_size, seq_size)
    state_h, state_c = net.zero_state(batch_size)

    # Transfer data to GPU
    state_h = state_h.to(device)
    state_c = state_c.to(device)
    for x, y in batches:
        iteration += 1

        # Tell it we are in training mode
        net.train()

        # Reset all gradients
        optimizer.zero_grad()

        # Transfer data to GPU
        x = torch.tensor(x).to(device)
        y = torch.tensor(y).to(device)

        logits, (state_h, state_c) = net(x, (state_h, state_c))
        loss = criterion(logits.transpose(1, 2), y)

        state_h = state_h.detach()
        state_c = state_c.detach()

        loss_value = loss.item()

        # Perform back-propagation
        loss.backward()

        _ = torch.nn.utils.clip_grad_norm_(
            net.parameters(), gradients_norm)

        optimizer.step()
        
        if iteration % 100 == 0:
            print('Epoch: {}/{}'.format(e, 200),
                  'Iteration: {}'.format(iteration),
                  'Loss: {}'.format(loss_value))

        if iteration % 1000 == 0:
            predict(device, net, initial_words, n_vocab,
                    vocab_to_int, int_to_vocab, top_k=5)
#             torch.save(net.state_dict(),
#                        'checkpoint_pt/model-{}.pth'.format(iteration))

Epoch: 0/200 Iteration: 100 Loss: 7.405266761779785
Epoch: 0/200 Iteration: 200 Loss: 7.68801212310791
Epoch: 1/200 Iteration: 300 Loss: 6.565629005432129
Epoch: 1/200 Iteration: 400 Loss: 5.581479549407959
Epoch: 1/200 Iteration: 500 Loss: 5.979805946350098
Epoch: 2/200 Iteration: 600 Loss: 5.549616813659668
Epoch: 2/200 Iteration: 700 Loss: 4.562647342681885
Epoch: 2/200 Iteration: 800 Loss: 4.640707015991211
Epoch: 3/200 Iteration: 900 Loss: 3.6189076900482178
Epoch: 3/200 Iteration: 1000 Loss: 3.763655185699463
Есть на больные препаратами наши друзья, 9 000 рублей. Мы просим вас сбор на оплату лечения позволяет вам жизнь натерпелся и даже в руки или медсестра А ведь если вы помощь высококвалифицированных блок химиотерапии. На 32 рождения ведет чистого сердца и нет возможности ребенка? семье! Пейли. В общем, а морфий с «бочкой», и волейболист. по удалению опухоли, на паспортном курс ЛФК в том всех кукол спать🙂Кстати и на раздумье Мы очень много сделано! 🆘 У малышки рак обоих констру

Epoch: 15/200 Iteration: 4100 Loss: 1.24717116355896
Epoch: 15/200 Iteration: 4200 Loss: 1.2984788417816162
Epoch: 15/200 Iteration: 4300 Loss: 1.236587643623352
Epoch: 16/200 Iteration: 4400 Loss: 1.3223400115966797
Epoch: 16/200 Iteration: 4500 Loss: 1.2474801540374756
Epoch: 17/200 Iteration: 4600 Loss: 1.006729245185852
Epoch: 17/200 Iteration: 4700 Loss: 1.3339279890060425
Epoch: 17/200 Iteration: 4800 Loss: 0.7480164170265198
Epoch: 18/200 Iteration: 4900 Loss: 1.267481803894043
Epoch: 18/200 Iteration: 5000 Loss: 1.2847856283187866
Есть на больные препаратами наши друзья, 9 000 рублей. Мы просим вас сбор на оплату лечения позволяет вам жизнь натерпелся и даже в руки или медсестра А ведь если вы помощь высококвалифицированных блок химиотерапии. На 32 рождения ведет чистого сердца и нет возможности ребенка? семье! Пейли. В общем, а морфий с «бочкой», и волейболист. по удалению опухоли, на паспортном курс ЛФК в том всех кукол спать🙂Кстати и на раздумье Мы очень много сделано! 🆘 У м

Epoch: 23/200 Iteration: 6400 Loss: 0.806170642375946
Epoch: 24/200 Iteration: 6500 Loss: 0.8711007237434387
Epoch: 24/200 Iteration: 6600 Loss: 0.7643405199050903
Epoch: 24/200 Iteration: 6700 Loss: 1.0386713743209839
Epoch: 25/200 Iteration: 6800 Loss: 0.7384764552116394
Epoch: 25/200 Iteration: 6900 Loss: 0.9054688811302185
Epoch: 25/200 Iteration: 7000 Loss: 0.937618613243103
Есть на больные препаратами наши друзья, 9 000 рублей. Мы просим вас сбор на оплату лечения позволяет вам жизнь натерпелся и даже в руки или медсестра А ведь если вы помощь высококвалифицированных блок химиотерапии. На 32 рождения ведет чистого сердца и нет возможности ребенка? семье! Пейли. В общем, а морфий с «бочкой», и волейболист. по удалению опухоли, на паспортном курс ЛФК в том всех кукол спать🙂Кстати и на раздумье Мы очень много сделано! 🆘 У малышки рак обоих конструкции уже все показатели крови. «Добрый вечер, друзья! Мы бесконечно Мы очень рады за вашу поддержку, за вашу поддержку, за жизнь Артемки. 

Epoch: 29/200 Iteration: 8100 Loss: 0.8016605377197266
Epoch: 30/200 Iteration: 8200 Loss: 0.7691028714179993
Epoch: 30/200 Iteration: 8300 Loss: 0.7745444178581238
Epoch: 31/200 Iteration: 8400 Loss: 0.7589635252952576
Epoch: 31/200 Iteration: 8500 Loss: 0.8119366765022278
Epoch: 31/200 Iteration: 8600 Loss: 0.7380120754241943
Epoch: 32/200 Iteration: 8700 Loss: 0.8711522817611694
Epoch: 32/200 Iteration: 8800 Loss: 0.6828420758247375
Epoch: 32/200 Iteration: 8900 Loss: 0.6679160594940186
Epoch: 33/200 Iteration: 9000 Loss: 0.49807870388031006
Есть на больные препаратами наши друзья, 9 000 рублей. Мы просим вас сбор на оплату лечения позволяет вам жизнь натерпелся и даже в руки или медсестра А ведь если вы помощь высококвалифицированных блок химиотерапии. На 32 рождения ведет чистого сердца и нет возможности ребенка? семье! Пейли. В общем, а морфий с «бочкой», и волейболист. по удалению опухоли, на паспортном курс ЛФК в том всех кукол спать🙂Кстати и на раздумье Мы очень много сделано!

Epoch: 37/200 Iteration: 10100 Loss: 0.8428534865379333
Epoch: 37/200 Iteration: 10200 Loss: 0.42195671796798706
Epoch: 38/200 Iteration: 10300 Loss: 0.7463955283164978
Epoch: 38/200 Iteration: 10400 Loss: 0.8241910338401794
Epoch: 38/200 Iteration: 10500 Loss: 0.6033732295036316
Epoch: 39/200 Iteration: 10600 Loss: 0.7671952843666077
Epoch: 39/200 Iteration: 10700 Loss: 0.7517375946044922
Epoch: 39/200 Iteration: 10800 Loss: 0.6518440842628479
Epoch: 40/200 Iteration: 10900 Loss: 0.6671502590179443
Epoch: 40/200 Iteration: 11000 Loss: 0.6437621116638184
Есть на больные препаратами наши друзья, 9 000 рублей. Мы просим вас сбор на оплату лечения позволяет вам жизнь натерпелся и даже в руки или медсестра А ведь если вы помощь высококвалифицированных блок химиотерапии. На 32 рождения ведет чистого сердца и нет возможности ребенка? семье! Пейли. В общем, а морфий с «бочкой», и волейболист. по удалению опухоли, на паспортном курс ЛФК в том всех кукол спать🙂Кстати и на раздумье Мы очень мног

Epoch: 44/200 Iteration: 12100 Loss: 0.6920291185379028
Epoch: 45/200 Iteration: 12200 Loss: 0.5299583673477173
Epoch: 45/200 Iteration: 12300 Loss: 0.6803966164588928
Epoch: 45/200 Iteration: 12400 Loss: 0.6592428684234619
Epoch: 46/200 Iteration: 12500 Loss: 0.6396356821060181
Epoch: 46/200 Iteration: 12600 Loss: 0.5672919750213623
Epoch: 47/200 Iteration: 12700 Loss: 0.4702817499637604
Epoch: 47/200 Iteration: 12800 Loss: 0.7315581440925598
Epoch: 47/200 Iteration: 12900 Loss: 0.36662396788597107
Epoch: 48/200 Iteration: 13000 Loss: 0.645576536655426
Есть на больные препаратами наши друзья, 9 000 рублей. Мы просим вас сбор на оплату лечения позволяет вам жизнь натерпелся и даже в руки или медсестра А ведь если вы помощь высококвалифицированных блок химиотерапии. На 32 рождения ведет чистого сердца и нет возможности ребенка? семье! Пейли. В общем, а морфий с «бочкой», и волейболист. по удалению опухоли, на паспортном курс ЛФК в том всех кукол спать🙂Кстати и на раздумье Мы очень много

Epoch: 48/200 Iteration: 13100 Loss: 0.7510092854499817
Epoch: 48/200 Iteration: 13200 Loss: 0.5255123972892761
Epoch: 49/200 Iteration: 13300 Loss: 0.6408633589744568
Epoch: 49/200 Iteration: 13400 Loss: 0.7043288946151733
Epoch: 49/200 Iteration: 13500 Loss: 0.5995252132415771
